This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'z_hqGJ9-bqMFJDC1eoHj'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
afx_x_sample = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-02&end_date=2017-01-02&api_key=' + API_KEY)
afx_x_sample

<Response [200]>

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
afx_x_sample.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2017-01-02',
  'end_date': '2017-01-02',
  'frequency': 'daily',
  'data': [['2017-01-02',
    34.99,
    35.94,
    34.99,
    35.8,
    None,
    44700.0,
    1590561.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017

In [5]:
# Request AFX_X stock data from quandl API for all of 2017
afx_x_2017 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY)
afx_x_2017

<Response [200]>

#### 2. Convert the returned JSON object into a Python dictionary.

In [6]:
# Convert response into a dictionary
afx_x_2017_dict = afx_x_2017.json()

In [7]:
# All of the relevent daily stock data is in afx_x_2017_dict['dataset_data']['data']
afx_x_2017_data = afx_x_2017_dict['dataset_data']['data']

# afx_x_2017_data is a list of lists
# Each list within the outer list represents a separate day
# Show the first 2 lists (days) of afx_x_2017_data
afx_x_2017_data[:2]

[['2017-12-29',
  51.76,
  51.94,
  51.45,
  51.76,
  None,
  34640.0,
  1792304.0,
  None,
  None,
  None],
 ['2017-12-28',
  51.65,
  51.82,
  51.43,
  51.6,
  None,
  40660.0,
  2099024.0,
  None,
  None,
  None]]

#### Extract Data

In [8]:
#Create empty lists to store daily data
open_list = []
close_list = []
one_day_change_list = []
volume_list = []

# First entry will not make sense because this represents the change from one day to the next. So set first entry to zero.
two_day_change_list = [0]

In [9]:
# Extract daily values and store in the appropriate list
# Each i represents a separate day
# For each day, 'open_price' is index 1, 'high' is index 2, 'low' is index 3, 'close' is index 4, and 'volume' is index 6
for i in range(len(afx_x_2017_data)):
    open_price = afx_x_2017_data[i][1]
    high = afx_x_2017_data[i][2]
    low = afx_x_2017_data[i][3]
    close = afx_x_2017_data[i][4]
    volume = afx_x_2017_data[i][6]
    
    # Use isinstance() to check if a particular data entry exists for a day
    if isinstance(open_price, float):
        open_list.append(open_price)
    if isinstance(close, float):
        close_list.append(close)
    if isinstance(high, float) and isinstance(low, float):
        one_day_change = high - low
        one_day_change_list.append(one_day_change)
    if isinstance(volume, float):
        volume_list.append(volume)

    # two_day_change cannot be calculated for the first day
    if i != 0:
        two_day_change = close_list[i] - close_list[i - 1]
        two_day_change_list.append(two_day_change)

#### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [10]:
print('The highest opening price is: ' + '${:,.2f}'.format(max(open_list)))
print('The lowest opening price is: ' + '${:,.2f}'.format(min(open_list)))

The highest opening price is: $53.11
The lowest opening price is: $34.00


#### 4. What was the largest change in any one day (based on High and Low price)?

In [11]:
print('The largest change in any one day is: ' + '${:,.2f}'.format(max(one_day_change_list)))

The largest change in any one day is: $2.81


#### 5. What was the largest change between any two days (based on Closing Price)?

In [12]:
print('The largest change between any 2 consecutive days based on closing price is: ' + '${:,.2f}'.format(max(two_day_change_list)))

The largest change between any 2 consecutive days based on closing price is: $2.56


#### 6. What was the average daily trading volume during this year?

In [13]:
def avg(l):
    return sum(l)/len(l)

print('The average daily trading volume during the year is: ' + '${:,.2f}'.format(avg(volume_list)))   

The average daily trading volume during the year is: $89,124.34


#### 7. What was the median trading volume during this year?

In [14]:
def median(l):
    l_sorted = sorted(l)
    length = len(l)
    index = (length - 1) // 2
    
    if length % 2:
        return l_sorted[index]
    else:
        return (l_sorted[index] + l_sorted[index + 1])/2
    
print('The median daily trading volume during the year is: ' + '${:,.2f}'.format(median(volume_list)))

The median daily trading volume during the year is: $76,286.00
